QnA 에서 찾은 정보

1. Validation set도 최종 학습에 사용 가능.
2. 발생일시는 해당 일시에 화재가 발생했는지의 의미 -> 해당 인자로 인해 기후 정보등이 도출 -> 시간별, 계절별 요인을 도출하는 인자로 활용 가능
3. 인구수는 통계청 데이터를 기반( 2018년 기준 )
4. lw 변수들은 '건축법 시행령 제5조의5 제4호'에 정의된 건축물(다중이용건축물), 연면적 합계 3천제곱미더이상인 건축물, 다중이용업소등을 대상으로 조사된 값들입니다. 해당 변수들은 0~5점 까지의 분포이며, 높을수록 좋습니다. 가스와 전기 에너지 사용량 데이터는 건축데이터 민간개방시스템(http://open.eais.go.kr/) 에서 수급된 데이터로서, 값이 없는 것들은 해당 시스템에 대상이 아닌 건축물이라는 의미이며, 0인 경우는 값이 실제 0인 경우입니다. 
5. bldng_cnt 의미 : 해당 데이터는 지적(땅의 1필지)를 기준으로 작성된 데이터 입니다. 즉 1필지의 땅에 여러채의 건물이 있는 경우가 있습니다.(예: 아파트) 그 때 건물수를 건물채수라고 함.
6. lnd_ar (토지면적) 의미 : 토지면적은 해당 필지의 면적 입니다. 토지면적이 0인 경우는 값이 누락된 경우라고 생각하시면 됩니다. 
7. 강수량같은 경우 0인 경우는 강우가 있기는 했지만 그 양이 거의 측정되지 않은 경우이며, NA의 경우는 실제 전혀 강우가 없는 경우임.
8. 













## 0. Load Dependencies & Data 

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.listdir("/content/drive/My Drive/gimhae")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['PJT002_test.csv', 'PJT002_validation.csv', 'PJT002_train.csv']

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [5]:
work_dir = '/content/drive/My Drive/gimhae/'

train_set = pd.read_csv((work_dir + 'PJT002_train.csv'),encoding='utf-8' )
validation_set = pd.read_csv((work_dir + 'PJT002_validation.csv'),encoding='utf-8' )
# 일단 무거우니까 test_set 놔 둠.
# test_set = pd.read_csv((work_dir + '/' + 'PJT002_test.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,172,173,174,175,176,177,178,179) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
train_set[train_set['fr_yn']=='Y'].shape

(7657, 180)

In [7]:
train_set['lw_13101010']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
59194   NaN
59195   NaN
59196   NaN
59197   NaN
59198   NaN
Name: lw_13101010, Length: 59199, dtype: float64

## 1. EDA

In [8]:
train_set.head()

,dt_of_fr,fr_yn,bldng_us,bldng_archtctr,bldng_cnt,bldng_ar,ttl_ar,lnd_ar,dt_of_athrztn,ttl_grnd_flr,ttl_dwn_flr,bldng_us_clssfctn,tmprtr,prcpttn,wnd_spd,wnd_drctn,hmdt,gas_engry_us_201401,ele_engry_us_201401,gas_engry_us_201402,ele_engry_us_201402,gas_engry_us_201403,ele_engry_us_201403,gas_engry_us_201404,ele_engry_us_201404,gas_engry_us_201405,ele_engry_us_201405,gas_engry_us_201406,ele_engry_us_201406,gas_engry_us_201407,ele_engry_us_201407,gas_engry_us_201408,ele_engry_us_201408,gas_engry_us_201409,ele_engry_us_201409,gas_engry_us_201410,ele_engry_us_201410,gas_engry_us_201411,ele_engry_us_201411,gas_engry_us_201412,...,lw_13101211,lw_13101310,lw_13101410,lw_13111010,lw_13111110,lw_13121010,lw_13121011,lw_13131010,lw_13131110,lw_13141010,lw_13141011,jmk,id,rgnl_ar_nm,rgnl_ar_nm2,lnd_us_sttn_nm,rd_sd_nm,emd_nm,hm_cnt,fr_sttn_dstnc,bldng_ar_prc,fr_wthr_fclt_dstnc,fr_mn_cnt,mlt_us_yn,cctv_dstnc,fr_wthr_fclt_in_100m,cctv_in_100m,tbc_rtl_str_dstnc,sft_emrgnc_bll_dstnc,ahsm_dstnc,no_tbc_zn_dstnc,bldng_cnt_in_50m,trgt_crtr,fr_fghtng_fclt_spcl_css_5_yn,fr_fghtng_fclt_spcl_css_6_yn,us_yn,dngrs_thng_yn,slf_fr_brgd_yn,blk_dngrs_thng_mnfctr_yn,cltrl_hrtg_yn
0,2017-10-20 05:54:00,Y,단독주택,블록구조,3,69.42,69.42,0.0,1977,1.0,0.0,주거용,10.7,NaN,0.9,200.0,96.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,대,106536,자연녹지지역,지정되지않음,단독,세로한면(불),경상남도 창녕군 창녕읍,17360.0,4547,NaN,133,137.0,N,112,0,0,1891,29231,11322,88,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-09-30 08:26:15,N,NaN,NaN,3,46.29,46.29,0.0,NaN,1.0,0.0,NaN,19.5,NaN,0.5,20.0,74.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,대,86366,계획관리지역,지정되지않음,단독,세로한면(가),경상남도 의령군 가례면,1791.0,6388,122581.0,489,85.0,N,818,0,0,4533,20480,3369,7727,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-10-30 14:57:00,Y,공동주택,철근콘크리트구조,1,583.80,2516.76,1446.0,2.00011e+07,5.0,0.0,주거용,15.5,NaN,2.0,90.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,답,107560,제2종일반주거지역,지정되지않음,연립,중로한면,경상남도 창녕군 창녕읍,17285.0,3340,618105.0,143,137.0,N,165,0,0,277,31197,12451,72,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-06-14 05:23:58,N,단독주택,일반목구조,2,48.92,48.92,0.0,1936,1.0,0.0,주거용,20.6,NaN,0.4,0.0,91.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,대,97550,제1종일반주거지역,지정되지않음,단독,세로한면(불),경상남도 창원시의창구 대산면,7327.0,179,719542.0,1585,176.0,N,131,0,0,438,4270,407,508,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-04-22 05:38:59,N,NaN,NaN,2,0.00,0.00,0.0,NaN,NaN,NaN,NaN,12.7,NaN,0.6,360.0,89.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,임,130501,자연환경보전지역,지정되지않음,자연림,세로한면(불),경상남도 창녕군 창녕읍,17278.0,4822,NaN,603,137.0,N,686,0,0,1702,29778,12487,707,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59199 entries, 0 to 59198
Columns: 180 entries, dt_of_fr to cltrl_hrtg_yn
dtypes: float64(147), int64(12), object(21)
memory usage: 81.3+ MB



#####  일단 3가지 맡은 변수를 나누고, 할당했습니다. 

In [0]:
id = train_set.loc[:,'id']
ele = train_set.loc[:,'ele_engry_us_201401':'ele_engry_us_201812']
ele = pd.concat([ele,id], axis = 1)
gas = train_set.loc[:,'gas_engry_us_201401':'gas_engry_us_201812']
gas = pd.concat([gas,id], axis = 1)
lw = train_set.loc[:,'lw_13101010':'lw_13141011']
lw = pd.concat([lw,id], axis = 1)

In [14]:
print(ele.shape)
print(gas.shape)
print(lw.shape)

(59199, 120)
(59199, 120)
(59199, 15)


In [15]:
ele.iloc[:,0].value_counts(dropna=False) # 14년 1월 전기 사용량
# 전체 데이터가 59199인데, 대충 결측이거나 0에 가까운 데이터가 58600이 넘습니다. 

0.0        31189
NaN        27581
273.0          3
3318.0         3
7203.0         3
           ...  
95382.0        1
19755.0        1
35454.0        1
1440.0         1
69657.0        1
Name: ele_engry_us_201401, Length: 381, dtype: int64

In [16]:
gas.iloc[:,0].value_counts(dropna=False) # 14년 1월 가스 사용량
# 이쪽도 마찬가지입니다.

0.0           31332
NaN           27581
25665200.0        5
50752.0           4
987056.0          4
              ...  
3500.0            1
16284.0           1
2004.0            1
27965.0           1
127194.0          1
Name: gas_engry_us_201401, Length: 255, dtype: int64

In [17]:
lw.iloc[:,0].value_counts(dropna=False) # lw1 : 복도, 계단, 출입구의 성능 유지여부
lw.iloc[:,0].value_counts(dropna=True).values.sum() # lw1 : 복도, 계단, 출입구의 성능 유지여부 중 결측이 아닌 값의 개수. 302입니다.

302

In [18]:
train_set.loc[train_set['mlt_us_yn']=='Y',:].shape[0] 
# mlt_us_yn열이 다중이용시설 포함여부기에 어떻게 써먹을 수 없을까 하고 일단 개수를 봤는데 당장 lw보다 개수가 적습니다. 
# 아마도 lw변수는 다중이용시설 뿐 아니라 면적, 다중이용업소등을 모두 포함하기 때문인 것으로 추정됩니다.
# Q&A에 질문은 올려두겠습니다.

189

## 1-2) EDA_2 ( 맡은 변수)

In [19]:
# 내가 만질 변수_1
print(ele.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59199 entries, 0 to 59198
Columns: 120 entries, ele_engry_us_201401 to id
dtypes: float64(119), int64(1)
memory usage: 54.2 MB
None


*0은 없는 값, NaN은 결측*

In [20]:
# 내가 만질 변수_2
print(gas.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59199 entries, 0 to 59198
Columns: 120 entries, gas_engry_us_201401 to id
dtypes: float64(119), int64(1)
memory usage: 54.2 MB
None


*마찬가지 0은 없는 값, NaN은 결측*

In [24]:
# 변수가 너무 많이서 대체 뭘 어떻게 해야 할지 감이 안옵니다. 일단 데이터를 수집한 기간이 같으니 평균 강수량, 가스사용량 열을 생성해서 뭘 해보겠습니다.
ele['tot'] = ele.mean(1, skipna=True)
print(ele['tot'].sort_values(ascending = False).head(20)

12786    2.087865e+08
12645    2.087863e+08
12658    2.087862e+08
12637    2.087862e+08
12664    2.087861e+08
56616    1.725066e+08
56619    1.725065e+08
56622    1.725064e+08
56615    1.725063e+08
56626    1.725062e+08
56624    1.725061e+08
36378    1.691603e+08
30341    1.691602e+08
33079    1.691602e+08
36377    1.691600e+08
36381    1.691597e+08
36380    1.691597e+08
2694     9.074334e+07
2714     9.074291e+07
2704     9.074289e+07
Name: tot, dtype: float64

In [35]:
# 총 전력 사용량이 동일한 애들이 많아서, 같은 건물인가 싶어 id value를 찍어봤습니다. 
ele['tot'].sort_values(ascending = False).head(20).keys()
print(ele.loc[[12664,12637,12786,12645,12658],]) # 다릅니다.

       ele_engry_us_201401  gas_engry_us_201402  ...      id           tot
12664                  0.0           30016800.0  ...   72136  2.087861e+08
12637                  0.0           30016800.0  ...   88029  2.087862e+08
12786                  0.0           30016800.0  ...  126509  2.087865e+08
12645                  0.0           30016800.0  ...  104888  2.087863e+08
12658                  0.0           30016800.0  ...   90812  2.087862e+08

[5 rows x 121 columns]


In [43]:
# 일단 이상하니 주소를 찍었습니다. 
print(train_set.loc[train_set['id'].isin([72136,88029,126509,104888,90812]),'emd_nm']) #같습니다.

# 다른 것도 찍어봅시다.
print(train_set.loc[train_set['id'].isin([72136,88029,126509,104888,90812]),['bldng_ar_prc','cctv_dstnc']]) #같습니다.

# 아주 높은 확률로 같은 건물인 것으로 추정됩니다.
# 아마도 불이 여러번 난 것이 아닐까 생각합니다. 

12637    경상남도 거제시 수양동
12645    경상남도 거제시 수양동
12658    경상남도 거제시 수양동
12664    경상남도 거제시 수양동
12786    경상남도 거제시 수양동
Name: emd_nm, dtype: object
       bldng_ar_prc  cctv_dstnc
12637     2024453.0          88
12645     2024453.0          88
12658     2024453.0          88
12664     2024453.0          88
12786     2024453.0          88


In [44]:
# 그러면 dt_of_fr을 찍어봅시다. 
print(train_set.loc[train_set['id'].isin([72136,88029,126509,104888,90812]),['dt_of_fr']])
# 예상대로입니다. 데이터가 굉장히 Tidy하지 않은 것을 확인할 수 있습니다. 
# 어떡하죠 이거. EDA전에 reshaping이 급해보입니다.
# 이 raw data를 그냥 쓰는 것은 의미가 없습니다. 중지합니다.

                  dt_of_fr
12637  2014-10-23 16:33:00
12645  2017-10-16 16:59:00
12658  2017-09-17 04:05:00
12664  2018-07-23 14:43:00
12786  2015-02-11 21:11:00
